# Request Elevation Data

In [1]:
import requests
import urllib
import pandas as pd

In [9]:
data = pd.read_csv('../DataFiles/locations_processed.csv')
data

,State,City,lat,lng,timezone,UTC
0,AK,ANCHORAGE,61.1508,-149.1091,America/Anchorage,-9
1,AK,FAIRBANKS,64.8353,-147.6534,America/Anchorage,-9
2,AK,JUNEAU,58.4546,-134.1739,America/Juneau,-9
3,AL,BIRMINGHAM,33.5277,-86.7987,America/Chicago,-6
4,AL,MOBILE,30.6783,-88.1162,America/Chicago,-6
...,...,...,...,...,...,...
135,WI,MADISON,43.0826,-89.3931,America/Chicago,-6
136,WI,MILWAUKEE,43.0642,-87.9673,America/Chicago,-6
137,WI,SHAWANO,44.7748,-88.5843,America/Chicago,-6
138,WV,CHARLESTON,38.3484,-81.6323,America/New_York,-5


In [10]:
# USGS Elevation Point Query Service
url = r'https://nationalmap.gov/epqs/pqs.php?'

# coordinates with known elevation 
lat = list(data['lat'])
lon = list(data['lng'])

# create data frame
df = pd.DataFrame({
    'lat': lat,
    'lon': lon
})

def elevation_function(df, lat_column, lon_column):
    """Query service using lat, lon. add the elevation values as a new column."""
    elevations = []
    for lat, lon in zip(df[lat_column], df[lon_column]):

        # define rest query params
        params = {
            'output': 'json',
            'x': lon,
            'y': lat,
            'units': 'Meters'
        }

        # format query string and return query value
        result = requests.get((url + urllib.parse.urlencode(params)))
        elevations.append(result.json()['USGS_Elevation_Point_Query_Service']['Elevation_Query']['Elevation'])

    df['elev_meters'] = elevations

elevation_function(df, 'lat', 'lon')
df.head()

,lat,lon,elev_meters
0,61.1508,-149.1091,236.45
1,64.8353,-147.6534,136.27
2,58.4546,-134.1739,570.19
3,33.5277,-86.7987,183.58
4,30.6783,-88.1162,9.08


In [11]:
data['Elevation']=df['elev_meters']
data

,State,City,lat,lng,timezone,UTC,Elevation
0,AK,ANCHORAGE,61.1508,-149.1091,America/Anchorage,-9,236.45
1,AK,FAIRBANKS,64.8353,-147.6534,America/Anchorage,-9,136.27
2,AK,JUNEAU,58.4546,-134.1739,America/Juneau,-9,570.19
3,AL,BIRMINGHAM,33.5277,-86.7987,America/Chicago,-6,183.58
4,AL,MOBILE,30.6783,-88.1162,America/Chicago,-6,9.08
...,...,...,...,...,...,...,...
135,WI,MADISON,43.0826,-89.3931,America/Chicago,-6,257.99
136,WI,MILWAUKEE,43.0642,-87.9673,America/Chicago,-6,216.83
137,WI,SHAWANO,44.7748,-88.5843,America/Chicago,-6,249.33
138,WV,CHARLESTON,38.3484,-81.6323,America/New_York,-5,182.17


In [12]:
data.to_csv('cities_coord_time_alt.csv')